# Healy chapter 5 figures (2022-09-02)

_by A. Maurits van der Veen_  

_Modification history:_  
_2022-08-20 - Initial version_  

This notebook provides python code to parallel the R code in Healy's Data Visualization book. 

It uses the `plotnine` module in python, which replicates most of ggplot.


## 0. General preparation

The code relies on several python modules, which may need to be installed first. To do so, uncomment the next X code snippets.

- `plotnine` is the main module replicating 
- `mizani` provides axis label formatting
- `pyreadr` reads R-format datasets
- `adjustText` is used to move text around to avoid overlapping (alternative to ggrepel)


In [ ]:
# !pip install plotnine


In [ ]:
# !pip install mizani


In [ ]:
# !pip install pyreadr


In [ ]:
# !pip install adjustText


In [ ]:
import math
import numpy as np

import pandas as pd
# import geopandas as gpd  # Not necessary until chapter 7

import matplotlib.pyplot as plt

from plotnine import *  # alternative: import plotnine as p9 and always use prefix
from mizani.formatters import currency_format, percent_format

import adjustText

import pyreadr


## Chapter 5 - Graph tables, add labels, make notes


The headings below correspond to chapter sections in the book, with the same titles. Where the title is purely R-specific (as in section 5.1 here), I have added an adjustment in italics.

In [ ]:
localfolder = '/Users/yourname/Downloads/'  # Change to local path


In [ ]:
# Retrieve & load data from the website associated with Healy's book

remotefolder = 'https://github.com/kjhealy/socviz/blob/master/data/'

datasetname = 'gss_sm'
targetfile = datasetname + '.rda'
pyreadr.download_file(remotefolder + targetfile + '?raw=true', localfolder + targetfile)
newdata = pyreadr.read_r(localfolder + targetfile)
gss_sm = newdata[datasetname]

datasetname = 'organdata'
targetfile = datasetname + '.rda'
pyreadr.download_file(remotefolder + targetfile + '?raw=true', localfolder + targetfile)
newdata = pyreadr.read_r(localfolder + targetfile)
organdata = newdata[datasetname]

datasetname = 'elections_historic'
targetfile = datasetname + '.rda'
pyreadr.download_file(remotefolder + targetfile + '?raw=true', localfolder + targetfile)
newdata = pyreadr.read_r(localfolder + targetfile)
elections_historic = newdata[datasetname]


In [ ]:
# in gss_sm, replace Null religion by 'NA': now NA will get plotted, as in the book

religions = ['Protestant', 'Catholic', 'Jewish', 'None', 'Other']

gss_smY = gss_sm.copy()
gss_smY['religion'] = gss_smY.apply(lambda row: 'NA' if row['religion'] not in religions else row['religion'], axis=1)

### 5.1 Use pipes to summarize data _(no pipes in python, use pandas' groupby instead)_

Note: in pandas, we generally use groupby instead.

In [ ]:
# Keep only bigregion and religion variables, and get the size of each cell
rel_by_region = gss_smY[['bigregion', 'religion']].groupby(['bigregion', 'religion']).size().reset_index(name='N')

# Get subtotals by region, to prepare for getting proportions
regionsize = dict()
for regionname, regiongroup in rel_by_region.groupby('bigregion'):
    regionsize[regionname] = regiongroup['N'].sum()

# Get proportion, and then multiply and round to get clean percentage values
rel_by_region['freq'] = rel_by_region.apply(lambda row: row['N']/regionsize[row['bigregion']], axis=1)
rel_by_region['pct'] = round(rel_by_region['freq'] * 100, 0).astype(int)


In [ ]:
# Compare result against table in Healy (pp. 98-99).
# Note that sorting is again done alphabetically by region and then religion

rel_by_region

In [ ]:
# Figure 5.2

p = ggplot(rel_by_region, aes(x = 'bigregion', y = 'pct', fill = 'religion'))

p + geom_col(position = 'dodge2') + \
    labs(x = 'Region', y = 'Percent', fill = 'Religion') + \
    theme(legend_position = 'top')

In [ ]:
# Set up the data and basic aesthetic, to re-use repeatedly below

p = ggplot(rel_by_region, 
           aes(x = 'religion', y = 'pct', fill = 'religion'))


In [ ]:
# Figure 5.3
# Note the use of nrow and of the figure_size option 
# to come closer to replicating the look in the book

# Note also that instead of putting \ after every + to allow us to run across
# multiple lines, we can simply enclose the whole expression in parentheses

(p + geom_col(position = 'dodge2') + 
     labs(x = '', y = 'Percent', fill = 'Religion') + 
     guides(fill = False) + 
     coord_flip() +
     facet_wrap('bigregion', nrow=1) +
     theme(figure_size = (10, 3)))

### 5.2 Continuous variables by group or category

Here we shift to using a new dataset.

In [ ]:
# Extract a sample of columns and rows to inspect what the data look like
cols2keep = ['country', 'year', 'donors', 'pop', 'pop_dens', 'gdp']

organdata_sample = organdata[cols2keep].sample(n = 10)
organdata_sample

In [ ]:
# Figure 5.4

p = ggplot(data = organdata, 
           mapping = aes(x = 'year', y = 'donors'))

(p + geom_point())

In [ ]:
# Figure 5.5

p + geom_line(aes(group = 'country')) + \
    facet_wrap('country')

In [ ]:
# Figure 5.6 -- Note that x labels are not automatically made diagonal and thus overlap

p = ggplot(data = organdata, 
           mapping = aes(x = 'country', y = 'donors'))

p + geom_boxplot()

In [ ]:
# Figure 5.6 again, now using a theme option
# to make country names display diagonally

p + geom_boxplot() +\
    theme(axis_text_x = element_text(rotation=45, hjust=1))

In [ ]:
# Figure 5.7

p + geom_boxplot() + coord_flip()

In [ ]:
# In preparation for figure 5.8, sort countries by mean number of donors 
# (averaged across years)

# In ggplot it is possible to do this on the fly
# Note: plotnine has aes(x='reorder(var2, value)' option, usable when
# we have a simple 'sort x by value of y' setup (not here, since we use averages)

meandonors = []
for countryname, countrydata in organdata.groupby('country'):
    meandonors.append((countrydata['donors'].mean(), countryname))
    
countrylist = [country for _, country in sorted(meandonors)]
country_cat = pd.Categorical(organdata['country'], categories=countrylist)

# assign to a new column in the DataFrame
organdata = organdata.assign(country_cat = country_cat)


In [ ]:
# Figure 5.8 

p = ggplot(data = organdata, 
           mapping = aes(x = 'country_cat', y = 'donors'))

p + geom_boxplot() +\
    labs(x = '') +\
    coord_flip()

In [ ]:
# Figure 5.8 redone with geom_violin, as suggested in the text

p + geom_violin() +\
    labs(x = '') +\
    coord_flip()

In [ ]:
# Figure 5.9

p = ggplot(data = organdata, 
           mapping = aes(x = 'country_cat', y = 'donors', fill='world'))

p + geom_boxplot() + \
    labs(x = '') + \
    coord_flip() + \
    theme(legend_position = 'top')

In [ ]:
# Figure 5.10

p = ggplot(data = organdata, mapping = aes(x = 'country_cat', 
                                           y = 'donors', 
                                           color = 'world'))

(p + geom_point() + 
     labs(x = '') + 
     coord_flip() + 
     theme(legend_position = 'top'))

In [ ]:
# Figure 5.11

(p + geom_jitter() + 
     labs(x = '') + 
     coord_flip() + 
     theme(legend_position = 'top'))

In [ ]:
# Figure 5.12

p + geom_jitter(position = position_jitter(width=0.15)) + \
    labs(x = '') + \
    coord_flip() + \
    theme(legend_position = 'top')

In [ ]:
# Check column names of the organdata dataframe
organdata.columns

In [ ]:
# Generate a dataframe with country-level summaries
# In the book, this is done on p. 110, using a pipeline
# Here, we use pandas' groupby
groupcols = ['consent_law', 'country']

# Get means and standard deviations for variables of interest.
# Note: For some reason 'gdp' and 'cerebvas' get dropped if we do it all once!
# This should not happen, but is easy enough to work around by doing them separately
cols2keep = ['donors', 'health', 'roads']  
organdata_grouped = organdata[groupcols + cols2keep].groupby(groupcols)
organdata_means = organdata_grouped.aggregate(np.mean).reset_index()
organdata_stds = organdata_grouped.aggregate(np.std).reset_index()

cols2keep2 = ['gdp', 'cerebvas']
organdata_grouped2 = organdata[groupcols + cols2keep2].groupby(groupcols)
organdata_means2 = organdata_grouped2.aggregate(np.mean).reset_index()
organdata_stds2 = organdata_grouped2.aggregate(np.std).reset_index()

# Now merge the two sets of variables-of-interest together
organdata_means = organdata_means.merge(organdata_means2, left_on=groupcols,
                                                          right_on=groupcols)
organdata_stds = organdata_stds.merge(organdata_stds2, left_on=groupcols,
                                                          right_on=groupcols)

# Finally, merge the two summary dfs together, adjusting names by adding suffixes
by_country = organdata_means.merge(organdata_stds, left_on=groupcols,
                                          right_on=groupcols, suffixes=('_mean', '_std'))


In [ ]:
# Add the sorted donor list again to control ordering
country_cat = pd.Categorical(by_country['country'], categories=countrylist)

# assign to a new column in the DataFrame
by_country = by_country.assign(country_cat = country_cat)


In [ ]:
# Display by_country (comparable to table on p. 112)
by_country

In [ ]:
# Figure 5.13
# Note that in the book the top and bottom dots are on the overall frame of the figure, 
# which is clunky. Plotnine's default is better

p = ggplot(data = by_country,
           mapping = aes(x = 'donors_mean', y = 'country_cat', color='consent_law'))

(p + geom_point(size = 3) +
     labs(x = 'Donor procurement rate',
          y = '',
          color = 'Consent law') +
     theme(legend_position = 'top')
)

In [ ]:
# Figure 5.14

p = ggplot(data = by_country,
           mapping = aes(x = 'donors_mean', y = 'country_cat'))

(p + geom_point(size = 3) +
     labs(x = 'Donor procurement rate',
          y = '') +
     facet_wrap('consent_law', scales = 'free_y', ncol = 1) +
     theme(figure_size = (4, 6))
)

In [ ]:
# Figure 5.15

by_country['donors_low'] = by_country['donors_mean'] - by_country['donors_std']
by_country['donors_high'] = by_country['donors_mean'] + by_country['donors_std']

p = ggplot(data = by_country,
           mapping = aes(x = 'country_cat', y = 'donors_mean'))

(p + geom_pointrange(mapping = aes(ymin = 'donors_low',
                                   ymax = 'donors_high')) +
     labs(x = '',
          y = 'Donor procurement rate') +
     coord_flip()
)

### 5.3 Plot text directly

In [ ]:
# Figure 5.16

p = ggplot(data = by_country, mapping = aes(x = 'roads_mean', y = 'donors_mean'))

p + geom_point() + geom_text(mapping = aes(label = 'country'))

In [ ]:
# Figure 5.17

p + geom_point() + geom_text(mapping = aes(label = 'country'),
                             ha = 'left')

In [ ]:
# Figure 5.18
# Note: instead of using ggrepel, we use the adjust_text option in plotnine,
# which requires the module adjustText (imported at the top)
# Also, we boost the top of the x scale a bit to make sure the text labels don't get cut off
# To avoid unnecessary legend on the right, the show_legend = False is necessary in geom_label

p = ggplot(elections_historic, aes(x = 'popular_pct', y = 'ec_pct'))

# Define some text strings
p_title = 'Presidential elections: Popular & Electoral College margins'
p_subtitle = '1824-2016'
p_caption = 'Data for 2016 are provisional'
x_label = "Winner's share of popular vote"
y_label = "Winner's share of Electoral College votes"

(p + geom_hline(yintercept = 0.5, size = 1.4, color = 'gray') +
     geom_vline(xintercept = 0.5, size = 1.4, color = 'gray') +
     geom_point() +  
     geom_label(aes(label = "winner_label", size = 5),
                adjust_text = {'expand_points': (1.5, 1.5),
                               'arrowprops': {'arrowstyle': '-'}},
                show_legend = False) + 
     scale_x_continuous(limits = (0.30, 0.64), labels = percent_format()) +
     scale_y_continuous(labels = percent_format()) +
     labs(x = x_label, y = y_label,
          title = p_title + '\n' + p_subtitle, caption = p_caption) +
     theme(figure_size = (10, 10))
)

### 5.4 Label outliers

In [ ]:
# Figure 5.19a

p = ggplot(data = by_country,
           mapping = aes(x = 'gdp_mean', y = 'health_mean'))

p + geom_point() + \
    geom_label(data = by_country[by_country['gdp_mean'] > 25000],
               mapping = aes(label = "country", size = 5),
               adjust_text = {'expand_points': (1.5, 1.5),
                           'arrowprops': {'arrowstyle': '-'}},
               show_legend = False)


In [ ]:
# Figure 5.19b

p + geom_point() + \
    geom_label(data = by_country[(by_country['gdp_mean'] > 25000) | 
                                 (by_country['health_mean'] < 1500) |
                                 (by_country['country'] == 'Belgium')],
               mapping = aes(label = "country", size = 5),
               adjust_text = {'expand_points': (1.5, 1.5),
                           'arrowprops': {'arrowstyle': '-'}},
               show_legend = False)


In [ ]:
# New variable for figure 5.20
# Note: the text says 1998, but to duplicate the graph, we need to use 1988

organdata['ind'] = organdata.apply(lambda row: row['ccode'] in ('Ita', 'Spa') and \
                                               row['year'] is not None and row['year'].year >= 1988, axis = 1)

In [ ]:
# Figure 5.20

p = ggplot(data = organdata,
           mapping = aes(x = 'roads', y = 'donors', color = 'ind'))

(p + geom_point() +
     geom_label(data = organdata[organdata['ind']],
                mapping = aes(label = "ccode", size = 5),
                adjust_text = {'expand_points': (1.5, 1.5),
                               'arrowprops': {'arrowstyle': '-'}},
                show_legend = False) +
     guides(color = False)
)

### 5.5 Write and draw in the plot area

In [ ]:
# Figure 5.21

p = ggplot(data = organdata,
           mapping = aes(x = 'roads', y = 'donors'))

(p + geom_point() +
     annotate(geom = 'text', x = 91, y = 33, 
              label = 'A surprisingly high\nrecovery rate',
              ha = 'left')
)

In [ ]:
# Figure 5.22

(p + geom_point() +
     annotate(geom = 'rect', xmin = 125, xmax = 155, ymin = 30, ymax = 35, 
              fill = 'red', alpha = 0.2) +
     annotate(geom = 'text', x = 157, y = 33, 
              label = 'A surprisingly high\nrecovery rate',
              ha = 'left')
)

### 5.6 Understanding scales, guides, and themes

In [ ]:
# Figure 5.23

p = ggplot(data = organdata,
           mapping = aes(x = 'roads',
                         y = 'donors',
                         color = 'world'))

p + geom_point()

In [ ]:
# Figure 5.25

(p + geom_point() +
     scale_x_log10() + 
     scale_y_continuous(breaks = (5, 15, 25),
                        labels = ('Five', 'Fifteen', 'Twentyfive'))
)

In [ ]:
# Figure 5.26

(p + geom_point() +
     scale_color_discrete(labels = ('Corporatist', 'Liberal', 'Social Democratic', 'Unclassified')) +
     labs(x = 'Road deaths', y = 'Donor procurement', color = 'Welfare state') 
)

In [ ]:
# Figure 5.27

(p + geom_point() +
     labs(x = 'Road deaths', y = 'Donor procurement') +
     guides(color = False)
)

### This completes the figures for chapter 5